In [1]:
import fasttext
import os
import json
from utils import fixText, resetLabelLv2, resetLabelLv3

TEMP_DIR = './tmp/'

def getConfig():
    config = {}
    with open('./config.json', 'r') as f:
        s = f.read()
        config = json.loads(s)
    return config

# 读入数据集的json文件，处理成fasttext接口使用的"文本__label__标签"形式，以txt文件存储
def readDataSet(path):
    with open(path, 'r') as f:
        s = f.read()
        data_set = json.loads(s)
    set1 = []
    set2 = []
    set3 = []
    for id in list(data_set.keys()):
        set1.append('__label__' + data_set[id]['tag_level_1'] + ' ' + fixText(data_set[id]['text']))
        set2.append('__label__' + resetLabelLv2(data_set[id]['tag_level_2']) + ' ' + fixText(data_set[id]['text']))
        set3.append('__label__' + resetLabelLv3(data_set[id]['tag_level_3']) + ' ' + fixText(data_set[id]['text']))
        
    try:
        os.mkdir(TEMP_DIR)
    except:
        pass
    with open(TEMP_DIR + 'set1.txt', 'w') as f:
        for l in set1:
            f.write(l + '\n')
    with open(TEMP_DIR + 'set2.txt', 'w') as f:
        for l in set2:
            f.write(l + '\n')
    with open(TEMP_DIR + 'set3.txt', 'w') as f:
        for l in set3:
            f.write(l + '\n')

In [2]:
config = getConfig()
readDataSet(config['data_path'] + config['data_set_name'])
# model_label1 = fasttext.train_supervised(
#     input = TEMP_DIR + '',
#     lr = config['lr'],
#     dim = config['hidden_dim'],
#     epoch = config['epoch']
# )
model_label2 = fasttext.train_supervised(
    input = TEMP_DIR + 'set2.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)
model_label3 = fasttext.train_supervised(
    input = TEMP_DIR + 'set3.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)

try:
    os.remove(TEMP_DIR + 'set1.txt')
    os.remove(TEMP_DIR + 'set2.txt')
    os.remove(TEMP_DIR + 'set3.txt')
    os.removedirs(TEMP_DIR)  
except:
    pass

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.742 seconds.
Prefix dict has been built successfully.
Read 17M words
Number of words:  233058
Number of labels: 459
Progress: 100.0% words/sec/thread:  128026 lr:  0.000000 avg.loss:  0.970484 ETA:   0h 0m 0s
Read 17M words
Number of words:  233069
Number of labels: 970
Progress: 100.0% words/sec/thread:   64089 lr:  0.000000 avg.loss:  1.827442 ETA:   0h 0m 0s


In [3]:
with open(config['data_path'] + 'test_set.json', 'r') as f:
    content = json.loads(f.read())

total = 0
true_tag1 = 0
true_tag2 = 0
true_tag3 = 0
for id in content.keys():
    total += 1
    text = content[id]['text']
    text = fixText(text)
    tag1 = content[id]['tag_level_1']
    tag2 = resetLabelLv2(content[id]['tag_level_2'])
    tag3 = resetLabelLv3(content[id]['tag_level_3'])
    # predict1 = model_label1.predict(text)[0][0]
    predict2 = model_label2.predict(text)[0][0]
    predict3 = model_label3.predict(text)[0][0]
    # if(predict1.replace('__label__', '') == tag1):
    #     true_tag1 += 1
    if(predict2.replace('__label__', '') == tag2):
        true_tag2 += 1
    if(predict3.replace('__label__', '') == tag3):
        true_tag3 += 1
print('tag level 1 accurate: {}% ({}/{})'.format(true_tag1 * 100 / total, true_tag1, total))
print('tag level 2 accurate: {}% ({}/{})'.format(true_tag2 * 100 / total, true_tag2, total))
print('tag level 3 accurate: {}% ({}/{})'.format(true_tag3 * 100 / total, true_tag3, total))

tag level 1 accurate: 0.0% (0/38330)
tag level 2 accurate: 75.8022436733629% (29055/38330)
tag level 3 accurate: 65.5778763370728% (25136/38330)


lr = 0.1 epoch = 25

标签数量

In [8]:
# print('tag 1 num: ', len(model_label1.labels))
print('tag 2 num: ', len(model_label2.labels))
print('tag 3 num: ', len(model_label3.labels))

tag 2 num:  459
tag 3 num:  970
